# So searching stellar parameters on the MAST portal is a major pain. 

## We can query our planet candidates with our output file "Sector_NN_PlanetCandidates_TLS.txt"

## I looked at the source code from the transitleastsquares function "catalog_info". It's basically using astroquery.mast's "Catalogs" function with specific outputs. 

We can grab a number of parameters, specifically:
- TESS Mag
- Visual Mag
- Effective Temperature
- Distance (pc)
- Mass (solar)
- Radius (solar)
- logg 


## Let's go through an example of what this query and results look like

In [45]:
#Needed for handling lists, arrays, strings, importing data and general math
import numpy as np

import astropy

# Old query tool
# from transitleastsquares import catalog_info #specific outputs from MAST catalog

# New query tool
from astroquery.mast import Catalogs

In [46]:
candidateIDs = np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[0]

In [49]:
print(len(candidateIDs))
for x in range(1):
    print(" ")
    
    ID = str(candidateIDs[x])[:-2]
    result = Catalogs.query_criteria(catalog="Tic", ID=int(ID))#.as_array() #<---used in TLS code
    
# pretty simple, right? Almost makes you wonder why I didn't do this before...

135
 


# Let's print what this result looks

In [50]:
result

ID,version,HIP,TYC,UCAC,TWOMASS,SDSS,ALLWISE,GAIA,APASS,KIC,objType,typeSrc,ra,dec,POSflag,pmRA,e_pmRA,pmDEC,e_pmDEC,PMflag,plx,e_plx,PARflag,gallong,gallat,eclong,eclat,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,TWOMflag,prox,w1mag,e_w1mag,w2mag,e_w2mag,w3mag,e_w3mag,w4mag,e_w4mag,GAIAmag,e_GAIAmag,Tmag,e_Tmag,TESSflag,SPFlag,Teff,e_Teff,logg,e_logg,MH,e_MH,rad,e_rad,mass,e_mass,rho,e_rho,lumclass,lum,e_lum,d,e_d,ebv,e_ebv,numcont,contratio,disposition,duplicate_id,priority,eneg_EBV,epos_EBV,EBVflag,eneg_Mass,epos_Mass,eneg_Rad,epos_Rad,eneg_rho,epos_rho,eneg_logg,epos_logg,eneg_lum,epos_lum,eneg_dist,epos_dist,distflag,eneg_Teff,epos_Teff,TeffFlag,gaiabp,e_gaiabp,gaiarp,e_gaiarp,gaiaqflag,starchareFlag,VmagFlag,BmagFlag,splists,e_RA,e_Dec,RA_orig,Dec_orig,e_RA_orig,e_Dec_orig,raddflag,wdflag,objID
str9,str8,int64,str1,str10,str16,str1,str1,str19,str8,int64,str4,str7,float64,float64,str7,float64,float64,float64,float64,str5,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,int64,float64,str1,str1,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,float64,float64,str5,float64,float64,float64,float64,int64,str1,str5,str8,str13,float64,float64,float64,float64,float64,float64,int64,int64,str9
201256293,20190415,--,--,170-000119,00070543-5605045,--,--,4922569671300905088,27127986,--,STAR,tmgaia2,1.77270472555149,-56.084616395621,tmgaia2,339.791,0.0539479,-7.3202,0.053537,gaia2,40.0789,0.0381119,gaia2,315.310110503448,-59.965131573087,330.598774743206,-50.1969913500593,15.052,0.019,13.487,0.047,--,--,--,--,--,--,--,--,--,--,9.567,0.023,9.061,0.027,8.71,0.019,AAA-222-111-000-0-0,--,--,--,--,--,--,--,--,--,12.3151,0.00036,11.0832,0.00732927,cdwrf,cdwrf,3377.0,157.0,4.87991,0.000999322,--,--,0.343946,0.0102276,0.327211,0.0201929,8.04187,0.220599,DWARF,0.013861157,0.003420577,24.9328,0.02385,--,--,40,0.00961202849,--,--,0.00645083236,--,--,--,--,--,--,--,--,--,--,--,--,--,0.0238,0.0239,bj2018,--,--,cdwrf,13.7541,0.002081,11.1495,0.001339,1,--,cdwrf,apassdr9,cooldwarfs_v8,0.947839474404792,0.830290964498308,1.77532672032454,-56.0846479409284,0.0282318741185446,0.0278575766225298,1,0,508612472


# This result is a object class called Astropy Table
## (try printing type(result))
## This is kind of like a dictionary or a pandas dataframe
## If you know the name of the column (or key), you can grab its value

In [54]:
print (result['ra'])
print (result['dec'])
print (result['Tmag'])
print (result['Vmag'])
print (result['Teff'])
print (result['d'])
print (result['rad'])
print (result['mass'])
print (result['logg'])


       ra       
----------------
1.77270472555149
      dec       
----------------
-56.084616395621
  Tmag 
-------
11.0832
 Vmag 
------
13.487
 Teff 
------
3377.0
   d   
-------
24.9328
  rad   
--------
0.343946
  mass  
--------
0.327211
  logg 
-------
4.87991


# Now we can append these results to previous results from the BLS/TLS searches

In [101]:
import time
#from planet candidate list
# ids =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[0]
cams =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[1]
ccds=  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[2]
rstar =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[3]
mstar =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[4]
rplanet =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[5]
per =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[6]
transdur =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[7]
sdes =  np.transpose(np.loadtxt("Sector_01_PlanetCandidates_TLS.txt"))[8]

ras=[];decs=[];tmags=[];vmags=[];teffs=[];ds=[];plxs=[];rads=[];masses=[];loggs=[]
ids = []
start = time.time()
for x in range(len(candidateIDs)):    
    ID = str(candidateIDs[x])[:-2] #:-2 removes the .0 at the end of the TIC ID column
    result = Catalogs.query_criteria(catalog="Tic", ID=int(ID))
    ids.append(ID)
    ras.append(result['ra'][0])
    decs.append(result['dec'][0])
    tmags.append(result['Tmag'][0])
    vmags.append(result['Vmag'][0])
    teffs.append(result['Teff'][0])
    ds.append(result['d'][0])
    plxs.append(result['plx'][0])
    rads.append(result['rad'][0])
    masses.append(result['mass'][0])
    loggs.append(result['logg'][0])

end = time.time()

print ("runtime: ", (end-start)/60.0, " minutes")

runtime:  1.429842213789622  minutes


In [138]:
print(len(loggs), len(ids))
print(len(np.where(np.isnan(plxs)==True)[0])) 
# Some targets may have nans for some parameters or even zeros

135 135
19


/Users/felizd/anaconda2/envs/py3.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Warning: converting a masked element to nan.
  


In [139]:
# From Distance and visual magnitudes, I calculate the absolute magnitude:
absmag = np.array(vmags)-5*np.log10(np.array(ds)/10)
absmag=absmag.tolist() #needs to be list for saving

/Users/felizd/anaconda2/envs/py3.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Warning: converting a masked element to nan.
  


In [137]:
ls =np.array([ids,cams,ccds,tmags,plxs,ds,vmags,teffs,absmag,mstar,rstar,rplanet,per,transdur,sdes])
ls = ls.astype(np.float) #turns strings into floats
lsx = sorted(np.transpose(ls), key=lambda x: x[-1],reverse=True) 
#^^^ sort by SDE with highest at top of list

# print(lsx)

header='%1s\t%3s\t%11s\t%12s\t%18s\t%4s\t%4s\t%16s\t%6s\t%29s\t%13s\t%7s\t%26s\t%27s\t%3s'%('TIC ID','Cam','CCD','Tmag','pllx (mas)','distance (pc)','Vmag','Teff (K)','AbsMag','Mstar (solar)','Rstar (solar)','Rplanet','Period (d)','transitduration (d)','SDE')
np.savetxt("Sector_01_PlanetCandidates_TLS_For_Spreadsheet.txt",lsx,header=header,delimiter='\t',fmt="%-12s")#fmt='%s')



In [106]:
print (ids[0])

201256293
